## Import and Initialize the model

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch
import pandas as pd



roberta = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(roberta)
        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(roberta).to(device)



c:\Users\fidel\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\fidel\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\fidel\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fidel\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest. Caching files will still work but in a degraded 

## Analyze Text and output results as a Dataframe

In [24]:

# Initialize a dictionary to store overall sentiment counts
overall_sentiments = {'negative': 0, 'neutral': 0, 'positive': 0}

def AnalyzeSentiment(sentences):
    results = []
    global overall_sentiments

    for sentence in sentences:
        if len(sentence) >514:
          continue
        encoded_text = tokenizer(sentence, return_tensors='pt').to(device)
        output = model(**encoded_text)
        scores = output.logits[0].cpu().detach().numpy()
        scores = softmax(scores)
        max_index = scores.argmax()
        label = ['negative', 'neutral', 'positive'][max_index]

        # Update overall sentiment counts
        overall_sentiments[label] += 1

        result = {
            'text': sentence,
            'negative': scores[0],
            'neutral': scores[1],
            'positive': scores[2],
            'overall_sentiment': label
        }
        results.append(result)

    return pd.DataFrame(results)



## Test Sentences

In [9]:

# # Input sentences as a single array, including the tweet
# test_sentences = [
#     "The sun is shining brightly today.",
#     "I can't believe how delicious this meal is!",
#     "The traffic was unbearable this morning.",
#     "My cat always knows how to cheer me up.",
#     "This book kept me up all night, I couldn't put it down!",
#     "The internet connection is so slow, it's frustrating.",
#     "I'm excited to see what the future holds.",
#     "My favorite song came on the radio, and I couldn't help but dance.",
#     "Today is a lazy day, I'm just going to relax and watch movies.",
#     "I'm feeling a bit under the weather today.",
#     "The customer service representative was extremely helpful.",
#     "I'm feeling grateful for all the good things in my life.",
#     "The movie I watched last night was a masterpiece.",
#     "I'm so proud of my friend for accomplishing their goals.",
#     "I'm exhausted after a long day of work.",
#     "The flowers in the garden are blooming beautifully.",
#     "I'm feeling anxious about the upcoming presentation.",
#     "The new restaurant in town has amazing food.",
#     "I'm disappointed that my plans got canceled at the last minute.",
#     "My favorite team won the game, I'm ecstatic!",
#     "I'm feeling nostalgic listening to old songs.",
#     "The coffee at this café is too bitter for my taste.",
#     "I'm overwhelmed with all the work I have to do.",
#     "The beach is my happy place.",
#     "I absolutely loved the new movie!",
#     "The service at the restaurant was terrible.",
#     "I'm feeling quite happy today.",
#     "The weather is okay, nothing special.",
#     "I am extremely disappointed with the product.",
#     "What a wonderful experience!",
#     "The presentation was dull and uninteresting.",
#     "It's an average day, nothing much happened.",
#     "I am so grateful for all the support.",
#     "This is the worst vacation I've ever had.",
#     "I enjoyed the concert last night.",
#     "The new update has some issues.",
#     "Life is beautiful and full of opportunities.",
#     "The food was mediocre at best.",
#     "I am neither satisfied nor dissatisfied with the outcome.",
#     "Everything went smoothly during the event.",
#     "The movie was a complete waste of time.",
#     "I feel content with my current situation.",
#     "The customer service was very helpful and friendly.",
#     "I had a terrible time at the party.",
#     "Your company is killing it, great work.",
#     "I'm overjoyed by the unexpected news of receiving a promotion and a raise at work!",
#     "Unexpectedly finding money in an old jacket pocket made my day!",
#     "Even though it's raining, I'm grateful for the cozy day indoors.",
#     "Reaching inbox zero for the first time in months felt like a small victory.",
#     "The aroma of freshly baked cookies filled the house with warmth and comfort.",
#     "Receiving a thoughtful handwritten letter from a friend brought tears to my eyes.",
#     "Waking up to a beautiful sunrise reminded me of life's simple pleasures.",
#     "Despite the setbacks, I'm determined to make today a productive day.",
#     "Sharing a heartfelt conversation with a loved one made my heart feel full.",
#     "Completing a challenging workout left me feeling strong and accomplished.",
#     "Discovering a new favorite song instantly lifted my mood.",
#     "Spending time in nature always rejuvenates my spirit.",
#     "Finding the perfect gift for a friend's birthday filled me with excitement.",
#     "Receiving positive feedback on my work gave me a confidence boost.",
#     "Being surrounded by laughter and good company made for a memorable evening.",
#     "Catching up with an old friend over coffee was a much-needed pick-me-up.",
#     "Witnessing a random act of kindness restored my faith in humanity.",
#     "Finding a parking spot right in front of the store felt like a stroke of luck.",
#     "Indulging in my favorite comfort food brought a sense of contentment.",
#     "Unexpectedly running into an old friend at the grocery store was a pleasant surprise.",
#     "Receiving a compliment from a stranger made me feel appreciated.",
#     "Finding the perfect outfit for a special occasion boosted my confidence.",
#     "Watching a beautiful sunset reminded me to appreciate the beauty in everyday moments.",
#     "Bumping into an old teacher who remembered me fondly warmed my heart.",
#     "Receiving a hand-written thank-you note for a small gesture made my day.",
#     "Solving a difficult problem at work filled me with a sense of accomplishment.",
#     "Receiving unexpected praise from my boss boosted my morale.",
#     "Taking a leisurely stroll through the park helped clear my mind and reduce stress.",
#     "Unexpectedly receiving a gift from a coworker brightened my day.",
#     "Completing a DIY project I'd been putting off left me feeling proud of myself.",
#     "Hearing my favorite song on the radio instantly lifted my spirits.",
#     "Receiving a surprise visit from a friend I hadn't seen in years brought joy to my heart.",
#     "Finding a perfect spot to watch the sunrise filled me with awe and wonder.",
#     "Receiving a heartfelt compliment from a stranger made me feel appreciated.",
#     "Completing a challenging crossword puzzle gave me a sense of accomplishment.",
#     "Being surrounded by loved ones during difficult times provided me with strength and support.",
#     "Discovering a new hobby I'm passionate about brought excitement into my life.",
#     "Receiving unexpected praise for my cooking skills made me feel proud.",
#     "Getting a good night's sleep left me feeling refreshed and energized.",
#     "Receiving a spontaneous invitation to dinner from a friend lifted my spirits.",
#     "Receiving a spontaneous hug from a loved one filled me with warmth and happiness.",
#     "Finding a forgotten love letter from my partner brought tears of joy to my eyes.",
#     "Receiving an unexpected compliment from a stranger brightened my day.",
#     "Watching a heartwarming movie with a loved one brought tears of happiness to my eyes.",
#     "Receiving a surprise care package from a friend reminded me how much I'm loved.",
#     "Completing a challenging task at work left me feeling accomplished and proud.",
#     "Being greeted by my pet with unconditional love and excitement made my day.",
#      "The sun is shining brightly today.",
#     "I can't believe how delicious this meal is!",
#     "The traffic was unbearable this morning.",
#     "My cat always knows how to cheer me up.",
#     "This book kept me up all night, I couldn't put it down!",
#     "The internet connection is so slow, it's frustrating.",
#     "I'm excited to see what the future holds.",
#     "My favorite song came on the radio, and I couldn't help but dance.",
#     "Today is a lazy day, I'm just going to relax and watch movies.",
#     "I'm feeling a bit under the weather today.",
#     "The customer service representative was extremely helpful.",
#     "I'm feeling grateful for all the good things in my life.",
#     "The movie I watched last night was a masterpiece.",
#     "I'm so proud of my friend for accomplishing their goals.",
#     "I'm exhausted after a long day of work.",
#     "The flowers in the garden are blooming beautifully.",
#     "I'm feeling anxious about the upcoming presentation.",
#     "The new restaurant in town has amazing food.",
#     "I'm disappointed that my plans got canceled at the last minute.",
#     "My favorite team won the game, I'm ecstatic!",
#     "I'm feeling nostalgic listening to old songs.",
#     "The coffee at this café is too bitter for my taste.",
#     "I'm overwhelmed with all the work I have to do.",
#     "The beach is my happy place.",
#     "I absolutely loved the new movie!",
#     "The service at the restaurant was terrible.",
#     "I'm feeling quite happy today.",
#     "The weather is okay, nothing special.",
#     "I am extremely disappointed with the product.",
#     "What a wonderful experience!",
#     "The presentation was dull and uninteresting.",
#     "It's an average day, nothing much happened.",
#     "I am so grateful for all the support.",
#     "This is the worst vacation I've ever had.",
#     "I enjoyed the concert last night.",
#     "The new update has some issues.",
#     "Life is beautiful and full of opportunities.",
#     "The food was mediocre at best.",
#     "I am neither satisfied nor dissatisfied with the outcome.",
#     "Everything went smoothly during the event.",
#     "The movie was a complete waste of time.",
#     "I feel content with my current situation.",
#     "The customer service was very helpful and friendly.",
#     "I had a terrible time at the party.",
#     "Your company is killing it, great work.",
#     "Unexpectedly finding money in an old jacket pocket made my day!",
#     "Even though it's raining, I'm grateful for the cozy day indoors.",
#     "Reaching inbox zero for the first time in months felt like a small victory.",
#     "The aroma of freshly baked cookies filled the house with warmth and comfort.",
#     "Receiving a thoughtful handwritten letter from a friend brought tears to my eyes.",
#     "Waking up to a beautiful sunrise reminded me of life's simple pleasures.",
#     "Despite the setbacks, I'm determined to make today a productive day.",
#     "Sharing a heartfelt conversation with a loved one made my heart feel full.",
#     "Completing a challenging workout left me feeling strong and accomplished.",
#     "Discovering a new favorite song instantly lifted my mood.",
#     "Spending time in nature always rejuvenates my spirit.",
#     "Finding the perfect gift for a friend's birthday filled me with excitement.",
#     "Receiving positive feedback on my work gave me a confidence boost.",
#     "Being surrounded by laughter and good company made for a memorable evening.",
#     "Catching up with an old friend over coffee was a much-needed pick-me-up.",
#     "Witnessing a random act of kindness restored my faith in humanity.",
#     "Finding a parking spot right in front of the store felt like a stroke of luck.",
#     "Indulging in my favorite comfort food brought a sense of contentment.",
#     "Unexpectedly running into an old friend at the grocery store was a pleasant surprise.",
#     "Receiving a compliment from a stranger made me feel appreciated.",
#     "Finding the perfect outfit for a special occasion boosted my confidence.",
#     "Watching a beautiful sunset reminded me to appreciate the beauty in everyday moments.",
#     "Bumping into an old teacher who remembered me fondly warmed my heart.",
#     "Receiving a hand-written thank-you note for a small gesture made my day.",
#     "Solving a difficult problem at work filled me with a sense of accomplishment.",
#     "Receiving unexpected praise from my boss boosted my morale.",
#     "Taking a leisurely stroll through the park helped clear my mind and reduce stress.",
#     "Unexpectedly receiving a gift from a coworker brightened my day.",
#     "Completing a DIY project I'd been putting off left me feeling proud of myself.",
#     "Hearing my favorite song on the radio instantly lifted my spirits.",
#     "Receiving a surprise visit from a friend I hadn't seen in years brought joy to my heart.",
#     "Finding a perfect spot to watch the sunrise filled me with awe and wonder.",
#     "Receiving a heartfelt compliment from a stranger made me feel appreciated.",
#     "Completing a challenging crossword puzzle gave me a sense of accomplishment.",
#     "Being surrounded by loved ones during difficult times provided me with strength and support.",
#     "Discovering a new hobby I'm passionate about brought excitement into my life.",
#     "Receiving unexpected praise for my cooking skills made me feel proud.",
#     "Getting a good night's sleep left me feeling refreshed and energized.",
#     "Receiving a spontaneous invitation to dinner from a friend lifted my spirits.",
#     "Receiving a spontaneous hug from a loved one filled me with warmth and happiness.",
#     "Finding a forgotten love letter from my partner brought tears of joy to my eyes.",
#     "Receiving an unexpected compliment from a stranger brightened my day.",
#     "Watching a heartwarming movie with a loved one brought tears of happiness to my eyes.",
#     "Receiving a surprise care package from a friend reminded me how much I'm loved.",
#     "Completing a challenging task at work left me feeling accomplished and proud.",
#     "Being greeted by my pet with unconditional love and excitement made my day."
# ]


# # Analyze the sentiment of the entire array
# results = AnalyzeSentiment(test_sentences)

# # Display the results
# display(results)

# # Display the overall sentiment counts
# print("Overall Sentiment Counts:")
# print(overall_sentiments)

## Using the Youtube Data API to fetch comments from a Video

In [11]:
import googleapiclient.discovery
import googleapiclient.errors

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = "your dev key"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY
)

def fetchAllComments(video_id, pageToken=None):
    items = []
    maxCount = 2500

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=pageToken
        )

        response = request.execute()
        items.extend(response['items'])
        if len(items) >= maxCount:
            break
        if 'nextPageToken' in response:
            pageToken = response['nextPageToken']
        else:
            break

    return items

#full link https://www.youtube.com/watch?v=4_UDm-nCjeA
video_id = "ddTV12hErTc"
items = fetchAllComments(video_id)



## Output as DataFrame

In [17]:
comments = []

for item in items:
    comment = item['snippet']['topLevelComment']['snippet']
    comments.append([
        comment['authorDisplayName'],
        comment['publishedAt'],
        comment['updatedAt'],
        comment['likeCount'],
        comment['textDisplay']
    ])

df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])

display(df)


,author,published_at,updated_at,like_count,text
0,@A.R.77,2024-05-29T17:32:52Z,2024-05-29T17:32:52Z,0,That thing reminds me of those cheap digital t...
1,@simoninkin9090,2024-05-29T17:12:30Z,2024-05-29T17:12:30Z,0,I had a better view on teenage engineering bef...
2,@nichtnennenswert2894,2024-05-29T15:47:35Z,2024-05-29T15:47:35Z,0,"To this Point, i only knew the Pocket synthesi..."
3,@This_Is_Skooba,2024-05-29T14:34:22Z,2024-05-29T14:34:22Z,0,I&#39;ll never understand why this device exists
4,@ALT-vz3jn,2024-05-29T14:28:20Z,2024-05-29T14:28:20Z,0,I don’t want another device to be chained to.....
...,...,...,...,...,...
2495,@delbroox,2024-05-02T09:39:59Z,2024-05-02T09:39:59Z,0,Every company today is literally delivering un...
2496,@solidust573,2024-05-02T09:38:18Z,2024-05-02T09:38:18Z,0,MKBHD is trying to kill off everything. Folks ...
2497,@privatechannel8462,2024-05-02T09:36:25Z,2024-05-02T09:36:25Z,0,My wife loves her rabbit
2498,@jont2576,2024-05-02T09:36:15Z,2024-05-02T09:36:15Z,0,so its basically siri in an orange box.


## Analyze youtube comments

In [26]:
# print(df['text'])
youtubeAnalysisResults = AnalyzeSentiment(df['text'])
youtubeAnalysisResults.to_json("./Documents/youtubeAnalysisResults.json")

In [27]:
display(youtubeAnalysisResults)

,text,negative,neutral,positive,overall_sentiment
0,That thing reminds me of those cheap digital t...,0.625985,0.298193,0.075822,negative
1,I had a better view on teenage engineering bef...,0.187516,0.658549,0.153936,neutral
2,"To this Point, i only knew the Pocket synthesi...",0.032795,0.153609,0.813596,positive
3,I&#39;ll never understand why this device exists,0.928738,0.063043,0.008219,negative
4,I don’t want another device to be chained to.....,0.875495,0.113198,0.011307,negative
...,...,...,...,...,...
2408,Every company today is literally delivering un...,0.932844,0.059436,0.007720,negative
2409,MKBHD is trying to kill off everything. Folks ...,0.886697,0.104331,0.008972,negative
2410,My wife loves her rabbit,0.005299,0.036958,0.957743,positive
2411,so its basically siri in an orange box.,0.101311,0.827313,0.071376,neutral


In [28]:
print(overall_sentiments)

{'negative': 1202, 'neutral': 806, 'positive': 405}
